In [1]:
file = r"C:\Users\karimia\Drone-stoch\Data\5.xlsx"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
import math
import random
import itertools as it
import numpy as np
from pickle import LONG
import os
import pandas as pd
from timeit import default_timer as timer
import itertools
import random
import VRPSolverEasy as vrpse
import math
random.seed(110)


def dist(Locations,i,j):
    dist = ((Locations.iloc[i,0]-Locations.iloc[j,0])**2 + (Locations.iloc[i,1]-Locations.iloc[j,1])**2)**0.5
    return(dist)

In [3]:
def bnp_solver(Locations,C,D,F1,usage,n_bar,demand,Cap,B,s,t):

    def compute_euclidean_distance(x_i, x_j, y_i, y_j):
        return round(math.sqrt((x_i - x_j)**2 +(y_i - y_j)**2), 3)

    # Data
    cost_per_distance = Drone_cost
    begin_time = 0
    end_time = 5000
    nb_point = C+D

    coordinates = {str(i): (Locations.iloc[i,0],Locations.iloc[i,1]) for i in range(nb_point)}
    # Demands of points
    demands = [0]+demand[t,s]
    # Initialisation
    model = vrpse.Model()
    # Add vehicle type
    for f in range(F1):
        if n_bar[f] > 0:
            model.add_vehicle_type(id=f+1,start_point_id=0,end_point_id=0,name="V"+str(f),capacity=Cap[f],max_number=n_bar[f],var_cost_dist=cost_per_distance,tw_end=B[f])

    # Add depot
    model.add_depot(id=0, name="D1", tw_begin=0, tw_end=5000)

    coordinates_keys = list(coordinates.keys())
    # Add customers
    for i in range(1, nb_point):
        model.add_customer(id=i,penalty=pen, name=coordinates_keys[i],demand=demands[i],tw_begin=begin_time,tw_end=end_time)

    # Add links
    coordinates_values = list(coordinates.values())
    for i in range(0, nb_point):
        for j in range(i + 1, nb_point):
            dist = compute_euclidean_distance(coordinates_values[i][0],coordinates_values[j][0],coordinates_values[i][1],coordinates_values[j][1])
            model.add_link(start_point_id=i,end_point_id=j,distance=dist,time=usage*dist)

    solver_name = 'cplex'
    model.solve()
    model.export()
    Sol = model.solution.json['Solution']['bestSolutionValue']
#     route = model.solution.json['Solution'];
    return(Sol)

In [4]:
def dist(Locations,i,j):
        dist = ((Locations.iloc[i,0]-Locations.iloc[j,0])**2 + (Locations.iloc[i,1]-Locations.iloc[j,1])**2)**0.5
        return(dist)
Locations = pd.read_excel(file, header=0, index_col=0, sheet_name='Loc')
Demands = pd.read_excel(file, header=0, index_col=0, sheet_name='Dem')
Parm = pd.read_excel(file, header=0, index_col=0, sheet_name='Parm')
time_sub = 0;
time_mast = 0;
time_van = 0;
time_drone = 0;
M=100000000
rnd = np.random
rnd.seed(104)
D = 1 # number of depot
C = int(Parm.iloc[6,0]) #number of customers
T = 3;
S = 3;
F1 = 3 #type of drones
V = D+C
usage = 0.1
######changing
Cap = [75,100,130];
B = [10,15,20]
Drone_cost = 1;
buy_c = [500,750,1000]
pen = 300;
distC = {}
k=0
for i in range(V):
    for j in range(V):
        distC[i,j] = dist(Locations,i,j) 
        k+=1

dem = {0:list(Demands.iloc[:,0])}

#############################################################################################
mu, sigma = 0.5, 0.1 # mean and standard deviation
demand ={};
demt={}
for t in range(T):
    demt[t] = Demands.iloc[:,0]*np.random.normal(1, 0.1, 1)[0]
    for s in range(S):
        delta = np.random.normal(mu, sigma, 1)[0]
        inc = [1, 1+delta , 1+2*delta]
        dum_var = list(map(int, (list(demt[t]*inc[s]))))
        demand[t,s] = dum_var

sum_dem = [];
for t in range(T):
    sum_dem +=[sum(demand[t,2])];
maxd = max(sum_dem)

numb = [int(np.ceil(maxd/Cap[0])),int(np.ceil(maxd/Cap[1])),int(np.ceil(maxd/Cap[2]))]
n_bar = {0: 0, 1: 0, 2: numb[2]}

In [5]:
start = timer()

def compute_euclidean_distance(x_i, x_j, y_i, y_j):
    return round(math.sqrt((x_i - x_j)**2 +(y_i - y_j)**2), 3)

# Data
cost_per_distance = Drone_cost
begin_time = 0
end_time = 5000
nb_point = C+D

coordinates = {str(i): (Locations.iloc[i,0],Locations.iloc[i,1]) for i in range(nb_point)}
# Demands of points
demands = [0]+demand[t,s]
# Initialisation
model = vrpse.Model()
# Add vehicle type
for f in range(F1):
    if n_bar[f] > 0:
        model.add_vehicle_type(id=f+1,start_point_id=0,end_point_id=0,name="V"+str(f),capacity=Cap[f],max_number=n_bar[f],var_cost_dist=cost_per_distance,tw_end=B[f])

# Add depot
model.add_depot(id=0, name="D1", tw_begin=0, tw_end=5000)

coordinates_keys = list(coordinates.keys())
# Add customers
for i in range(1, nb_point):
    model.add_customer(id=i, name=coordinates_keys[i],demand=demands[i],tw_begin=begin_time,tw_end=end_time)

# Add links
coordinates_values = list(coordinates.values())
for i in range(0, nb_point):
    for j in range(i + 1, nb_point):
        dist = compute_euclidean_distance(coordinates_values[i][0],coordinates_values[j][0],coordinates_values[i][1],coordinates_values[j][1])
        model.add_link(start_point_id=i,end_point_id=j,distance=dist,time=0)

solver_name = 'cplex'
model.solve()
model.export()
Sol = model.solution.json['Solution']['bestSolutionValue']

print(timer()-start)

1.9999607000000008


In [6]:
T_s =0;
start = timer()
n_bar = {0: 0, 1: 0, 2: numb[2]}
Upper_bound = 0;
s= 0
for t in range(T):
    for se in range(S):
        s += bnp_solver(Locations,C,D,F1,usage,n_bar,demand,Cap,B,se,t)/(S)
        print(s)
Upper_bound = s 
T_s += timer()-start
print(T_s)
start = timer()

19.319666666666667
38.63933333335265
62.12433333335265
81.44400000001932
104.92900000001931
128.4140000000193
147.733666666686
167.05333333335267
190.53833333335265
14.361701899999998


In [7]:
names = [];
ObjValue={};
n1 = {};
n2 = {};
n3 = {}
for i in range(numb[0]):
    for j in range(numb[1]):
        for ii in range(numb[2]):
            name = str(str(i)+str(j)+str(ii))
            ObjValue[name] = Upper_bound; 
            names += [name]
            n1[name] = i;
            n2[name] = j;
            n3[name] = ii;

In [8]:
T_m = 0;
start = timer()
ms = Model('gmaster');
ms.Params.LogToConsole = 0
ms.modelSense = GRB.MINIMIZE;
g = ms.addVars(names, vtype=GRB.BINARY)
beta = ms.addVars(range(1), vtype=GRB.CONTINUOUS)
ms.setObjective(quicksum((buy_c[0]*n1[i]+buy_c[1]*n2[i]+buy_c[2]*n3[i])*g[i] for i in names)+beta[0])
ms.addConstr(quicksum(g[i] for i in names)==1);
ms.addConstr(quicksum((n1[i]*Cap[0]+n2[i]*Cap[1]+n3[i]*Cap[2])*g[i] for i in names) >= sum(demand[0,0]));
ms.addConstr(beta[0]>= quicksum(ObjValue[i]*g[i] for i in names));
ms.optimize()
T_m += timer()-start
start = timer()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 678219
Academic license 678219 - for non-commercial use only - registered to ka___@student.unimelb.edu.au
Academic license 678219 - for non-commercial use only - registered to ka___@student.unimelb.edu.au


In [9]:
activeg = [i for i in names if g[i].x == 1]
n_bar={}
n_bar[0] = n1[activeg[0]];
n_bar[1] = n2[activeg[0]];
n_bar[2] = n3[activeg[0]];

In [10]:
Obj = 0
for t in range(T):
    for s in range(S):
        Obj += bnp_solver(Locations,C,D,F1,usage,n_bar,demand,Cap,B,s,t)/S

ObjValue[activeg[0]] = Obj
T_s += timer()-start
print(T_s)
start = timer()

51.6979204


In [11]:
U = 1000000
L = -1000000
M = 1000000;

itr = 0
while U-L>=0.01:
    itr += 1
    U = ObjValue[activeg[0]] + sum(buy_c[f]*n_bar[f] for f in range(F1));
    L = ms.ObjVal
    print(activeg[0])
    print(itr,U,L)
    
    ms.addConstr(beta[0]>= quicksum(ObjValue[i]*g[i] for i in names));
    ms.optimize()
    T_m += timer()-start
    start = timer()
    
    activeg = [i for i in names if g[i].x == 1]
    
    n_bar={}
    n_bar[0] = n1[activeg[0]];
    n_bar[1] = n2[activeg[0]];
    n_bar[2] = n3[activeg[0]];
    
    Obj = 0
    for t in range(T):
        for se in range(S):
            Obj += bnp_solver(Locations,C,D,F1,usage,n_bar,demand,Cap,B,se,t)/S
    ObjValue[activeg[0]] = Obj
    T_s += timer()-start
    start = timer()

100
1 1941.7096666663113 690.5383333333526
Academic license 678219 - for non-commercial use only - registered to ka___@student.unimelb.edu.au
010
2 1891.9943333331526 940.5383333333526
Academic license 678219 - for non-commercial use only - registered to ka___@student.unimelb.edu.au
001
3 1563.16766666655 1190.5383333333527
Academic license 678219 - for non-commercial use only - registered to ka___@student.unimelb.edu.au
200
4 1410.5173333333905 1190.5383333333527
Academic license 678219 - for non-commercial use only - registered to ka___@student.unimelb.edu.au
200
5 1410.5173333333905 1410.5173333333905
Academic license 678219 - for non-commercial use only - registered to ka___@student.unimelb.edu.au


In [13]:
print("time of solving master problem:  ", T_m, "time of solving sub problem:",T_s)

time of solving master problem:   1.4739222000000076 time of solving sub problem: 194.11989709999997


In [14]:
ObjValue

{'000': 190.53833333335265,
 '001': 563.1676666665501,
 '010': 1141.9943333331526,
 '011': 190.53833333335265,
 '100': 1441.7096666663113,
 '101': 190.53833333335265,
 '110': 190.53833333335265,
 '111': 190.53833333335265,
 '200': 410.51733333339047,
 '201': 190.53833333335265,
 '210': 190.53833333335265,
 '211': 190.53833333335265}

In [15]:
Cap

[75, 100, 130]